In [1]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random


import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.patches import Ellipse, Circle

In [14]:

hexagon_grid_table=pd.read_csv('hexagon_grid_table.csv',header=None,
                            names = ['Grid_id','v1_lng','v1_lat',\
                                    'v2_lng','v2_lat',\
                                    'v3_lng','v3_lat',\
                                    'v4_lng','v4_lat',\
                                    'v5_lng','v5_lat',\
                                    'v6_lng','v6_lat'])

hexagon_grid_table=hexagon_grid_table.dropna()



X=list(hexagon_grid_table['v1_lng'])+list(hexagon_grid_table['v2_lng'])+list(hexagon_grid_table['v3_lng'])+list(hexagon_grid_table['v4_lng'])+list(hexagon_grid_table['v5_lng'])+list(hexagon_grid_table['v6_lng'])

Y=list(hexagon_grid_table['v1_lat'])+list(hexagon_grid_table['v2_lat'])+list(hexagon_grid_table['v3_lat'])+list(hexagon_grid_table['v4_lat'])+list(hexagon_grid_table['v5_lat'])+list(hexagon_grid_table['v6_lat'])
# hexagon_grid_table
# hexagon_grid_table.iloc[0,1]
# for i in range(0,len(hexagon_grid_table)):
#     if((hexagon_grid_table.iloc[i,1]>hexagon_grid_table.iloc[i,3])|(hexagon_grid_table.iloc[i,2]>hexagon_grid_table.iloc[i,4])):
#         print('No1')
#         print(i)
#     if((hexagon_grid_table.iloc[i,3]<hexagon_grid_table.iloc[i,5])|(hexagon_grid_table.iloc[i,4]>hexagon_grid_table.iloc[i,6])):
#         print('No2')
#         print(i)
#     if((hexagon_grid_table.iloc[i,5]<hexagon_grid_table.iloc[i,7])|(hexagon_grid_table.iloc[i,6]>hexagon_grid_table.iloc[i,8])):
#         print('No3')
#         print(i)
#     if((hexagon_grid_table.iloc[i,7]<hexagon_grid_table.iloc[i,9])|(hexagon_grid_table.iloc[i,8]<hexagon_grid_table.iloc[i,10])):
#         print('No4')
#         print(i)
#     if((hexagon_grid_table.iloc[i,9]>hexagon_grid_table.iloc[i,11])|(hexagon_grid_table.iloc[i,10]<hexagon_grid_table.iloc[i,12])):
#         print('No5')
#         print(i)
#     if((hexagon_grid_table.iloc[i,11]>hexagon_grid_table.iloc[i,1])|(hexagon_grid_table.iloc[i,12]<hexagon_grid_table.iloc[i,2])):
#         print('No6')
#         print(i)

In [17]:
hexagon_grid_table=hexagon_grid_table.dropna()

hexagon_grid_table['c_lng']=hexagon_grid_table.apply(lambda x:(x['v1_lng']+x['v2_lng']+x['v3_lng']\
                                                               +x['v4_lng']+x['v5_lng']+x['v6_lng'])/6,axis=1)

hexagon_grid_table['c_lat']=hexagon_grid_table.apply(lambda x:(x['v1_lat']+x['v2_lat']+x['v3_lat']\
                                                               +x['v4_lat']+x['v5_lat']+x['v6_lat'])/6,axis=1)

hexagon_grid_table['Index']=0
hexagon_grid_table=hexagon_grid_table.sort_values(by=['c_lng','c_lat'])
for i in range(0,len(hexagon_grid_table)):
    hexagon_grid_table.iloc[i,15]=i+1
    
print(hexagon_grid_table)

               Grid_id     v1_lng    v1_lat     v2_lng    v2_lat     v3_lng  \
4803  3dfcc8aedf63ee3f  103.00165  29.98391  103.00559  29.98804  103.00352   
4982  a7e5c9728d156772  103.00972  29.97689  103.01366  29.98102  103.01159   
610   46d7d403e46e20c2  103.11772  30.29174  103.12168  30.29589  103.11961   
3126  5523ac18acc59fda  103.14961  30.30960  103.15358  30.31375  103.15150   
8423  fb665698fb213881  103.15150  30.31934  103.15547  30.32349  103.15340   
...                ...        ...       ...        ...       ...        ...   
4563  1f0bcd9362653a8f  116.49839  39.98117  116.50444  39.98540  116.50292   
2075  815eca1eea7182da  117.12810  40.14128  117.13423  40.14549  117.13275   
3927  3262a05e0aab92d1  117.94544  34.31559  117.95120  34.31966  117.94989   
203   101ef48879c8a933  118.04835  24.48930  118.05347  24.49300  118.05231   
4706  18ef3655da9320f3  120.36044  30.30984  120.36612  30.31372  120.36504   

        v3_lat     v4_lng    v4_lat     v5_lng    v

In [18]:
def IntersectsSegment(ptx,pty,sptx,spty,eptx,epty): #0 for no, 1 for yes
    #input: test point, start point , end point; all in format [lng, lat]
    
    if spty>pty and epty>pty: #segment over point
        return 0
    if spty<pty and epty<pty: #segment under point
        return 0
    if epty==pty: #end point is not counted
        return 0
    if sptx<ptx and eptx<ptx: #segment is on the left of test point
        return 0
    if spty==epty: #error
        print('error')
        return 0

    xseg=eptx-(eptx-sptx)*(epty-pty)/(epty-spty) #lng of intersection
    if xseg<ptx: #test point is on the right of segment
        return 0
    return 1  #test point intersects with segment

In [19]:
def WithinHex(ptx,pty,hexg): # 0 for no, 1 for yes
    #input test point and hexagon, all in format [lng, lat]
    
    if pty==hexg[2]:# test point is the lowest vertax
        if ptx==hexg[1]:
            return 1
        else:
            return 0
    
    if pty==hexg[8]:# test point is the highest vertax
        if ptx==hexg[7]:
            return 1
        else:
            return 0
        
    sc=0 # # of intersections
    for i in range(0,5):
        sc=sc+IntersectsSegment(ptx,pty,hexg[(2*i+1)],hexg[(2*i+2)],hexg[(2*i+3)],hexg[(2*i+4)])
    
    sc=sc+IntersectsSegment(ptx,pty,hexg[11],hexg[12],hexg[1],hexg[2])
    
    

    if sc%2==1:# if the # of intersections is odd
        return 1 # test point is in the hexagon
    else:# if the # of intersections is even
        return 0 # test point is not in the hexagon

In [20]:
ptx=hexagon_grid_table.iloc[0,3]-0.00001
pty=hexagon_grid_table.iloc[0,4]
hexg=hexagon_grid_table.iloc[0]
# IntersectsSegment(ptx,pty,hexg[9],hexg[10],hexg[11],hexg[12])
WithinHex(ptx,pty,hexg)


1